In [1]:
import numpy as np

In [2]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [3]:
print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


In [6]:
x[0, 0] # x[0][0]

array([[0.14601919, 0.94358366, 0.99241239, 0.48892532, 0.83173841,
        0.22319478, 0.41881   , 0.84027407, 0.54983029, 0.38639417,
        0.52395093, 0.2960137 , 0.4761555 , 0.11203922, 0.58088007,
        0.71001737, 0.96595279, 0.10655761, 0.32499695, 0.11357152,
        0.82310803, 0.03052781, 0.76426364, 0.41076461, 0.60301463,
        0.88847904, 0.86389856, 0.60593425],
       [0.90086296, 0.26879917, 0.27187542, 0.61175653, 0.93855722,
        0.21459937, 0.16749584, 0.11692449, 0.97922454, 0.53896253,
        0.4232036 , 0.46746823, 0.63048499, 0.01599357, 0.68488981,
        0.57427994, 0.70173123, 0.90969315, 0.02282145, 0.10246699,
        0.81949043, 0.0415407 , 0.751019  , 0.1400857 , 0.94895008,
        0.69309932, 0.61155029, 0.20619151],
       [0.48261522, 0.7678122 , 0.93688904, 0.41870175, 0.28218477,
        0.10311952, 0.64998098, 0.58129068, 0.15963626, 0.42416833,
        0.68612011, 0.1494176 , 0.07918982, 0.22662002, 0.40049125,
        0.0186219 , 0.3948

In [7]:
from common.util import im2col

In [9]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


### 7. 4. 3 합성곱 계층 구현하기

In [10]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

### 7. 4. 4 풀링 계층 구현하기

In [11]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

## 7. 5. 4 CNN 구현하기

In [14]:
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

In [15]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_szie']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_szsie - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_szie)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

In [17]:
def predict(self, x):
    for layer in self.layers.values():
         x = layer.forward(x)

    return x

def loss(self, x, t):
    y = self.predict(x)
    
    return self.last_layer.forward(y, t)

In [18]:
def gradient(self, x, t):
    self.loss(x, t)
    
    dout = 1
    dout = self.last_later.backward(dout)
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
        dout = layer.backward(dout)
        
    grads = {}
    grads['W1'] = self.layers['Conv1'].dW
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Affine1'].dW
    grads['b2'] = self.layers['Affine1'].db
    grads['W3'] = self.layers['Affine2'].dW
    grads['b3'] = self.layers['Affine2'].db
    
    return grads